# Conversion of Ckt-Bench101 graphs to Torch Geometric 2.6

The graphs from the Ckt-Bench101 dataset are provided in igraph format or torch-geometric format. However, the torch-geometric graphs were generated with an older torch-geometric version (< 2.0). Using this script, the igraph data can be converted to a newer torch-geometric format (2.6)

In [27]:
import pickle
import torch
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
from tqdm import tqdm

file = "./../data/ckt_bench_101.pkl"
with open(file, "rb") as f:
    graphs = pickle.load(f)

def igraph_to_torch_geometric(g_ig):
    g_tg = from_networkx(g_ig.to_networkx())
    type = F.one_hot(g_tg.type, 10).to(torch.float32)
    feat = g_tg.feat[:, None]
    x = torch.cat([type, feat], dim=1)
    g_tg = Data(x=x, edge_index=g_tg.edge_index)
    return g_tg

# Training set
print("Converting Training Set:")
train_graphs = []
pbar = tqdm(graphs[0])
for g in pbar:
    g_tg = igraph_to_torch_geometric(g[1])
    train_graphs.append(g_tg)

# Test set
print("Converting Test Set:")
test_graphs = []
pbar = tqdm(graphs[1])
for g in pbar:
    g_tg = igraph_to_torch_geometric(g[1])
    test_graphs.append(g_tg)

graphs = [train_graphs, test_graphs]

# Save to pickle
with open('./../data/ckt_bench_101_tg26.pkl', 'wb') as f:
    pickle.dump(graphs, f)


Converting Training Set:


100%|████████████████████████████████████████████████████████████████████████████| 9000/9000 [00:05<00:00, 1526.68it/s]


Converting Test Set:


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1197.28it/s]
